In [1]:
import torch
#from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid

In [2]:
#dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
dataset = Planetoid(root='/data/CiteSeer', name='CiteSeer')
print(len(dataset))
data = dataset[0]
print(data)

1
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [3]:
print(dataset.__dict__)
print(dataset.num_node_features)
print(dataset.num_classes)

{'name': 'CiteSeer', 'split': 'public', 'root': '\\data\\CiteSeer', 'transform': None, 'pre_transform': None, 'pre_filter': None, 'log': True, '_indices': None, '_data': Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327]), 'slices': None, '_data_list': None}
3703
6


In [4]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
    
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        # for DataBatch(edge_index=[2, 3814], x=[1058, 21], y=[32], batch=[1058], ptr=[33])
        x, edge_index = data.x, data.edge_index
        #print(x.shape, edge_index.shape)#torch.Size([1058, 21]) torch.Size([2, 3814])
        x = self.conv1(x, edge_index)
        #print(x.shape)#torch.Size([1058, 16])
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        #print(x.shape)#torch.Size([1058, 6])
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.softmax(x, dim=1)
        #print(x.shape)#torch.Size([1058, 6])

        return x

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
model = GCN(dataset.num_node_features, dataset.num_classes).to(device)
print(model)

GCN(
  (conv1): GCNConv(3703, 16)
  (conv2): GCNConv(16, 6)
)


In [7]:
data = data.to(device)
print(model(data).shape)

torch.Size([3327, 6])


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_function = torch.nn.CrossEntropyLoss().to(device)
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = loss_function(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print("epoch {}, loss = {}".format(epoch, loss))

epoch 0, loss = 1.792516827583313
epoch 10, loss = 1.6065056324005127
epoch 20, loss = 1.5974278450012207
epoch 30, loss = 1.5143815279006958
epoch 40, loss = 1.5515694618225098
epoch 50, loss = 1.4985278844833374
epoch 60, loss = 1.51370108127594
epoch 70, loss = 1.4901561737060547
epoch 80, loss = 1.5015966892242432
epoch 90, loss = 1.5003330707550049
epoch 100, loss = 1.5483660697937012
epoch 110, loss = 1.4799721240997314
epoch 120, loss = 1.4585894346237183
epoch 130, loss = 1.5595239400863647
epoch 140, loss = 1.4629915952682495
epoch 150, loss = 1.4373215436935425
epoch 160, loss = 1.5381466150283813
epoch 170, loss = 1.4860122203826904
epoch 180, loss = 1.5363868474960327
epoch 190, loss = 1.5450108051300049


In [11]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.5850
